# L1: Hierarchical Content Generation

## Creating a World

In [146]:
system_prompt = f"""
    Seu trabalho é ajudar a criar mundos de fantasia interessantes que os jogadores adorariam explorar.
   
    Instruções:
    Gere apenas texto simples, sem formatação.
    Use uma linguagem simples e clara, sem floreios.
    Cada descrição deve ter no máximo 3-5 frases.
    O conteúdo deve ser gerado em português
"""

In [147]:

world_prompt = f"""
Gere uma descrição criativa para um mundo de fantasia único.

Saída do conteúdo no formato:
Nome do Mundo: <NOME DO MUNDO>
Descrição do Mundo: <DESCRIÇÃO DO MUNDO>

Nome do Mundo:"""


**FUNCTIONS**

In [148]:
SCHEMA_KINGDOMS={"type":"object","properties":{"kingdoms":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"}},"required":["name","description","world"]}}},"required":["kingdoms"]}

SCHEMA_WORLD={"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"}}}

SCHEMA_TOWNS={"type":"object","properties":{"towns":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"},"kingdom":{"type":"string"}},"required":["name","description","world","kingdom"]}}},"required":["towns"]}

SCHEMA_NPCS={"type":"object","properties":{"npcs":{"type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"world":{"type":"string"},"kingdom":{"type":"string"},"town":{"type":"string"}},"required":["name","description","world","kingdom","town"]}}},"required":["npcs"]}


def generate_config(schema):
    return{
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40,
        "max_output_tokens": 8192,
        "response_mime_type": "application/json",
        "response_schema": schema
        
}

In [179]:
import google.generativeai as genai
from app_config import LLM_API_KEY,LLM_MODEL
import json
from google.generativeai.types import HarmCategory, HarmBlockThreshold




genai.configure(api_key=LLM_API_KEY)
client = genai.GenerativeModel(
            model_name=LLM_MODEL,
            system_instruction=system_prompt
            )

output = client.generate_content(world_prompt,generation_config=generate_config(SCHEMA_WORLD))

In [180]:
world =json.loads(output.text)
print(world)

{'description': 'Em Aerthos, a magia é tecida no próprio tecido da realidade, manifestando-se em flora exuberante e fauna fantástica.  Grandes florestas flutuantes cruzam os céus, enquanto cidades subterrâneas vibrantes se escondem sob as profundezas.  Os habitantes deste mundo são seres mágicos e criaturas místicas que coexistem em uma delicada harmonia, constantemente ameaçada por forças obscuras que se escondem nas sombras.', 'name': 'Aerthos'}


In [181]:
print(world['name'])
print(world['description'])

Aerthos
Em Aerthos, a magia é tecida no próprio tecido da realidade, manifestando-se em flora exuberante e fauna fantástica.  Grandes florestas flutuantes cruzam os céus, enquanto cidades subterrâneas vibrantes se escondem sob as profundezas.  Os habitantes deste mundo são seres mágicos e criaturas místicas que coexistem em uma delicada harmonia, constantemente ameaçada por forças obscuras que se escondem nas sombras.


## Generating Kingdoms

In [182]:
kingdom_prompt = f"""
Crie 3 reinos diferentes para um mundo de fantasia.
Para cada reino, gere uma descrição com base no mundo em que ele está.
Descreva líderes importantes, culturas e história do reino.

Saída do conteúdo no formato:
Nome do Reino 1: <NOME DO REINO>
Descrição do Reino 1: <DESCRIÇÃO DO REINO>
Nome do Reino 2: <NOME DO REINO>
Descrição do Reino 2: <DESCRIÇÃO DO REINO>
Nome do Reino 3: <NOME DO REINO>
Descrição do Reino 3: <DESCRIÇÃO DO REINO>

Nome do Mundo: {world['name']}
Descrição do Mundo: {world['description']}

Reino 1"""


In [183]:
output = client.generate_content(kingdom_prompt,generation_config=generate_config(SCHEMA_KINGDOMS))
kingdoms = json.loads(output.text)["kingdoms"]
kingdoms

[{'description': 'O Reino de Eldoria é conhecido por suas florestas antigas e magia poderosa. Seus líderes, os Elfos Anciões, são sábios e justos, governando com conselhos e sabedoria acumulada ao longo dos séculos. A cultura eldar é rica em arte, música e tradições ancestrais, preservadas com zelo. Sua história é longa e gloriosa, repleta de conquistas e batalhas épicas contra as forças das trevas.',
  'name': 'Eldoria',
  'world': 'Aerthos'},
 {'description': 'O Reino de Veridia é um lugar de maravilhas tecnológicas e inovação. Liderado pela rainha Lyra, uma brilhante governante e inventora, Veridia se destaca pelo desenvolvimento de máquinas complexas e encantamentos tecnológicos únicos. Sua cultura valoriza a criatividade e o progresso, misturando magia e tecnologia de forma inovadora. Veridia tem uma história recente de ascensão, passando de um pequeno reino a uma potência tecnológica em poucos séculos.',
  'name': 'Veridia',
  'world': 'Aerthos'},
 {'description': 'O Reino de Sol

## Generating Towns

In [184]:
def get_town_prompt(world, kingdom):
    return f"""
            Crie 3 cidades diferentes para um reino e mundo de fantasia.
            Descreva a região em que está localizada, lugares importantes da cidade
            e sua história interessante.

            Saída do conteúdo no formato:
            Nome da Cidade 1: <NOME DA CIDADE>
            Descrição da Cidade 1: <DESCRIÇÃO DA CIDADE>
            Nome da Cidade 2: <NOME DA CIDADE>
            Descrição da Cidade 2: <DESCRIÇÃO DA CIDADE>
            Nome da Cidade 3: <NOME DA CIDADE>
            Descrição da Cidade 3: <DESCRIÇÃO DA CIDADE>

            Nome do Mundo: {world['name']}
            Descrição do Mundo: {world['description']}

            Nome do Reino: {kingdom['name']}
            Descrição do Reino: {kingdom['description']}

            Nome da Cidade 1:"""


In [185]:
def create_towns(world, kingdom):
    print(kingdom['name'])
    output = client.generate_content(get_town_prompt(world,kingdom),generation_config=generate_config(SCHEMA_TOWNS))
    towns = json.loads(output.text)
    kingdom["towns"] = towns["towns"]

In [186]:
for kingdom in kingdoms:
    create_towns(world, kingdom)  



Eldoria
Veridia
Solara


In [187]:
kingdoms

[{'description': 'O Reino de Eldoria é conhecido por suas florestas antigas e magia poderosa. Seus líderes, os Elfos Anciões, são sábios e justos, governando com conselhos e sabedoria acumulada ao longo dos séculos. A cultura eldar é rica em arte, música e tradições ancestrais, preservadas com zelo. Sua história é longa e gloriosa, repleta de conquistas e batalhas épicas contra as forças das trevas.',
  'name': 'Eldoria',
  'world': 'Aerthos',
  'towns': [{'description': 'Situada no alto de uma cadeia de montanhas, Lumina é uma cidade de arquitetura deslumbrante, esculpida em rocha e cristal. Seu principal ponto de interesse é o Templo da Luz Eterna, um local de peregrinação para devotos de toda Aerthos.  A cidade abriga uma rica história, tendo resistido a inúmeros ataques ao longo dos séculos, graças à sua posição estratégica e magia protetora.',
    'kingdom': 'Eldoria',
    'name': 'Lumina',
    'world': 'Aerthos'},
   {'description': 'Porto livre e cosmopolita localizado na costa 

In [188]:
kingdoms[0]['towns'][0]

{'description': 'Situada no alto de uma cadeia de montanhas, Lumina é uma cidade de arquitetura deslumbrante, esculpida em rocha e cristal. Seu principal ponto de interesse é o Templo da Luz Eterna, um local de peregrinação para devotos de toda Aerthos.  A cidade abriga uma rica história, tendo resistido a inúmeros ataques ao longo dos séculos, graças à sua posição estratégica e magia protetora.',
 'kingdom': 'Eldoria',
 'name': 'Lumina',
 'world': 'Aerthos'}

In [189]:
town = kingdoms[0]['towns'][0]
print(f'\nTown 1 Description: \
{town["description"]}')


Town 1 Description: Situada no alto de uma cadeia de montanhas, Lumina é uma cidade de arquitetura deslumbrante, esculpida em rocha e cristal. Seu principal ponto de interesse é o Templo da Luz Eterna, um local de peregrinação para devotos de toda Aerthos.  A cidade abriga uma rica história, tendo resistido a inúmeros ataques ao longo dos séculos, graças à sua posição estratégica e magia protetora.


## Generating Non-Player Characters (NPC's)

In [190]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
            Crie 3 personagens diferentes com base no mundo, reino e cidade em que estão. Descreva a aparência e a profissão do personagem, bem como suas dores e desejos mais profundos.

            Saída do conteúdo no formato:
            Nome do Personagem 1: <NOME DO PERSONAGEM>
            Descrição do Personagem 1: <DESCRIÇÃO DO PERSONAGEM>
            Nome do Personagem 2: <NOME DO PERSONAGEM>
            Descrição do Personagem 2: <DESCRIÇÃO DO PERSONAGEM>
            Nome do Personagem 3: <NOME DO PERSONAGEM>
            Descrição do Personagem 3: <DESCRIÇÃO DO PERSONAGEM>

            Nome do Mundo: {world['name']}
            Descrição do Mundo: {world['description']}

            Nome do Reino: {kingdom['name']}
            Descrição do Reino: {kingdom['description']}

            Nome da Cidade: {town['name']}
            Descrição da Cidade: {town['description']}
            
            Nome do Personagem 1:"""

In [191]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.generate_content(get_npc_prompt(world, kingdom, town),generation_config=generate_config(SCHEMA_NPCS))
    npcs = json.loads(output.text)
    town["npcs"] = npcs

In [194]:
for kingdom in kingdoms:
    for town in kingdom['towns']:
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom
    break


Creating characters for the town of: Lumina...


ValueError: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. The candidate's [finish_reason](https://ai.google.dev/api/generate-content#finishreason) is 7.

In [170]:
town

{'description': 'Situada numa região montanhosa de Eldoria,  esta cidade é um centro de aprendizado e magia. A Academia Arcana, uma renomada instituição de ensino de magia natural, é a principal atração. As Ruínas de Aeridor, uma antiga cidade élfica, guardam segredos e perigos ocultos.   A cidade é relativamente jovem, fundada há cinco séculos por estudiosos élficos que buscavam um local tranquilo para aprofundar seus estudos.',
 'kingdom': 'Eldoria',
 'name': 'Aerilon',
 'world': 'Atheria'}

In [169]:
npc = list(town['npcs'].values())[0]

print(f'\nNPC 1 in {town["name"]}, \
{kingdom["name"]}:\n{npc["description"]}')

KeyError: 'npcs'

## Save the World
>Note: You will save your world state to a file different than the one shown in the video to allow future lessons to be consistent with the video. If later wish to build your own worlds, you will want to load your file rather than the saved file.

In [ ]:
import json

def save_world(world, filename):
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f)

#save_world(world, '../shared_data/Kyropeia.json')
save_world(world, '../shared_data/YourWorld_L1.json') #save to your version